In [2]:
import pandas as pd

In [3]:
github_url = 'https://github.com/DataTalksClub/llm-zoomcamp/blob/main/04-monitoring/data/results-gpt4o-mini.csv'
url = f'{github_url}?raw=1'
df = pd.read_csv(url)

In [4]:
df = df.iloc[:300]

In [5]:
from sentence_transformers import SentenceTransformer

model_name = 'multi-qa-mpnet-base-dot-v1'
embedding_model = SentenceTransformer(model_name)

modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/212 [00:00<?, ?B/s]

You try to use a model that was created with version 3.0.0.dev0, however, your version is 2.7.0. This might cause unexpected behavior or errors. In that case, try to update to the latest version.





README.md:   0%|          | 0.00/8.71k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [6]:
answer_llm = df.iloc[0].answer_llm


## Q1. Getting the embeddings model

In [7]:
first_embed = embedding_model.encode(answer_llm)
first_embed[0]

-0.42244658

In [8]:
def compute_similarity(record, model):
    answer_orig = record['answer_orig']
    answer_llm = record['answer_llm']
    
    v_llm = model.encode(answer_llm)
    v_orig = model.encode(answer_orig)
    
    return v_llm.dot(v_orig)

In [11]:
from tqdm.auto import tqdm

evaluations = []
result = df.to_dict(orient='records')

for record in tqdm(result):
    sim = compute_similarity(record, embedding_model)
    evaluations.append(sim)

  0%|          | 0/300 [00:00<?, ?it/s]

In [14]:
df['dot'] = evaluations
df['dot'].describe()

count    300.000000
mean      27.495996
std        6.384743
min        4.547925
25%       24.307841
50%       28.336864
75%       31.674304
max       39.476013
Name: dot, dtype: float64

In [17]:
import numpy as np
def compute_norm(v):
    norm = np.sqrt((v * v).sum())
    v_norm = v / norm
    return v_norm

def compute_similarity_1(record, model):
    answer_orig = record['answer_orig']
    answer_llm = record['answer_llm']
    
    v_llm = model.encode(answer_llm)
    v_llm_norm = compute_norm(v_llm)
    v_orig = model.encode(answer_orig)
    v_orig_norm = compute_norm(v_orig)
    
    return v_llm_norm.dot(v_orig_norm)



In [18]:
evaluations_1 = []


for record in tqdm(result):
    sim = compute_similarity_1(record, embedding_model)
    evaluations_1.append(sim)

  0%|          | 0/300 [00:00<?, ?it/s]

In [19]:
df['cosine'] = evaluations_1
df['cosine'].describe()

count    300.000000
mean       0.728392
std        0.157755
min        0.125357
25%        0.651273
50%        0.763761
75%        0.836235
max        0.958796
Name: cosine, dtype: float64

In [20]:
!pip install rouge

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [23]:
from rouge import Rouge
rouge_scorer = Rouge()

r = df.loc[10]
scores = rouge_scorer.get_scores(r['answer_llm'], r['answer_orig'])[0]

In [24]:
scores

{'rouge-1': {'r': 0.45454545454545453,
  'p': 0.45454545454545453,
  'f': 0.45454544954545456},
 'rouge-2': {'r': 0.21621621621621623,
  'p': 0.21621621621621623,
  'f': 0.21621621121621637},
 'rouge-l': {'r': 0.3939393939393939,
  'p': 0.3939393939393939,
  'f': 0.393939388939394}}

In [30]:
(scores['rouge-1']['f'] + scores['rouge-2']['f'] + scores['rouge-l']['f']) / 3

0.35490034990035496

In [32]:
evaluations_2 = []


for record in tqdm(result):
    scores = rouge_scorer.get_scores(record['answer_llm'], record['answer_orig'])[0]
    sim = [scores['rouge-1']['f'], scores['rouge-2']['f'] , scores['rouge-l']['f']]
    evaluations_2.append(sim)

  0%|          | 0/300 [00:00<?, ?it/s]

In [33]:
f_df = pd.DataFrame(evaluations_2)

In [35]:
f_df.columns=["rouge-1", "rouge-2", "rouge-l"]

In [38]:
f_df['rouge-2'].describe()

count    300.000000
mean       0.206965
std        0.153550
min        0.000000
25%        0.097809
50%        0.178671
75%        0.286181
max        0.739130
Name: rouge-2, dtype: float64